In [ ]:
import scaper
import random
%cd /nfs/home/pedro.lopes/data/audio_data/train
import audiofile as af
import IPython.display as ipd
from glob import glob
from os import path
import numpy as np
from pathlib import Path
import json
import tensorflow as tf

In [ ]:
signal, sampling_rate = af.read('/nfs/home/pedro.lopes/data/audio_data/train/augmented/val/sample_299998_vocals.wav')
ipd.Audio(signal,rate=44100)

In [ ]:
wav_encode = tf.audio.encode_wav(signal,sampling_rate)

In [ ]:
tf.io.write_file(
    '/nfs/home/pedro.lopes/data/test_audio_tf.wav', wav_encode, name=None
)

In [ ]:
buf = tf.io.read_file(
    '/nfs/home/pedro.lopes/data/test_audio_tf.wav', name=None
)
audio = tf.audio.decode_wav(buf)
audio[0].shape

In [ ]:
ipd.Audio(np.squeeze(audio[0].numpy()),rate=44100)

In [ ]:
stft_data = tf.signal.stft(signal, 
	frame_length=1024, 
	frame_step=512,
	fft_length=1024).numpy().T

json_path = '/home/pedro.lopes/pf/norm_data_full.json'
with open(json_path) as infile:
    norm_data = json.load(infile)

In [ ]:

num_zeros = int(np.ceil(x.shape[-1]/128)*128 - x.shape[-1])
pad = ([0,0],[0,num_zeros])
x = np.pad(x,pad,mode='constant', constant_values=0)


In [ ]:
signal, sampling_rate = af.read('/nfs/home/pedro.lopes/data/audio_data/train/augmented/train/sample_50_mix.wav')
signal = np.array([signal,signal])
stft_data = tf.signal.stft(signal, 
	frame_length=1024, 
	frame_step=512,
	fft_length=1024)
x_before = tf.keras.backend.permute_dimensions(stft_data,(0,2,1))
x_shape = tf.cast(tf.shape(x_before)[-1],tf.float64)
pad_len = tf.math.floor(tf.math.ceil(x_shape/128)*128 - x_shape)
pad = ([0,0],[0,0],[0,pad_len])
x = tf.pad(x_before,pad,mode='constant', constant_values=0)
X_mean = norm_data['X_min']
X_std = norm_data['X_max'] - norm_data['X_min']
def normalize(x,save=False):
    scaled_x = (x - np.mean(x))/(np.abs(np.std(x))+1e-8)

    if save:
      return scaled_x, np.mean(x), np.std(x)
    return scaled_x
def normalize_from_outer(x,x_mean,x_std):
  scaled_x = (x - x_mean)/(x_std+1e-8)
  return scaled_x
def preprocess(sample,x_mean,x_std):
  log_sample = np.log(np.abs(sample)+1e-7)
  mod_input = normalize_from_outer(log_sample,x_mean,x_std)
  return mod_input

def preprocess_tf(sample,x_mean,x_std):
  log_sample = tf.math.log(tf.math.abs(sample)+1e-7)
  mod_input = normalize_from_outer(log_sample,x_mean,x_std)
  return mod_input
x = preprocess_tf(x,X_mean,X_std)[:,0:512]


In [ ]:
x_split = tf.split(x,7,axis = -1)
tf.concat(x_split,axis=0)
#tf.keras.backend.transpose(x_split[:,0,:,0]) - x[0,:,0:128]

In [ ]:
tf.repeat([128],x_shape)

In [ ]:
pattern = "/home/pedro.lopes/data/audio_data/train/augmented/train/*"
names = [path.basename(x) for x in glob(pattern)]
%cd /home/pedro.lopes/pf/code/
names_corrected =[]
for name in names:
    splitted = 'train/' + name.rsplit('_',maxsplit=1)[0]
    names_corrected.append(splitted)
training_split = list(dict.fromkeys(names_corrected))

pattern = "/home/pedro.lopes/data/audio_data/train/augmented/val/*"
names = [path.basename(x) for x in glob(pattern)]
%cd /home/pedro.lopes/pf/code/
names_corrected =[]
for name in names:
    splitted = 'val/' + name.rsplit('_',maxsplit=1)[0]
    names_corrected.append(splitted)
validation_split = list(dict.fromkeys(names_corrected))

In [ ]:
dataset_path = '/home/pedro.lopes/data/audio_data/train/augmented/'
AUTOTUNE = tf.data.experimental.AUTOTUNE
def get_dataset(filepaths):
    vocal_filepaths = [dataset_path + x + '_vocals.wav' for x in filepaths]
    labels = [1 for x in filepaths]
    mix_filepaths = [dataset_path + x + '_mix.wav' for x in filepaths]
    X_ds = tf.data.Dataset.from_tensor_slices(mix_filepaths)
    y_ds = tf.data.Dataset.from_tensor_slices(labels)#tf.data.Dataset.from_tensor_slices(vocal_filepaths)
    return tf.data.Dataset.zip((X_ds, y_ds))


def load_audio(filepath, label):
    # Load 10 seconds of audio at 44.1kHz sample-rate
    audio = tf.io.read_file(filepath)
    audio, sample_rate = tf.audio.decode_wav(audio,
                                             desired_channels=-1,
                                             desired_samples=-1)
    return audio, label


def prepare_for_training(ds, shuffle_buffer_size=1024, batch_size=16):
    # Randomly shuffle (file_path, label) dataset
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    # Load and decode audio from file paths
    ds = ds.map(load_audio, num_parallel_calls=AUTOTUNE)
    # Repeat dataset forever
    ds = ds.repeat()
    # Prepare batches
    ds = ds.batch(batch_size)
    # Prefetch
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))    
with strategy.scope():
    ds = get_dataset(training_split)

In [ ]:
list(ds.map(load_audio).take(10).as_numpy_iterator())

In [ ]:
numbers = list(range(1,1335,1))
validation_size = 0.2
validation_items = random.sample(numbers,k=int(len(numbers)*validation_size))
training_items = [x for x in numbers if x not in validation_items]
training_items

In [ ]:
validation_items.sort()
validation_items

In [ ]:
for i in training_items:
    name = 'sample_' + str(i) + '.wav'
    vocal_src = 'vocals/' + name
    vocal_dest = 'train/vocals/' + name
    acc_src = 'acc/' + name
    acc_dest = 'train/acc/' + name
    mix_src = 'mix/' + name
    mix_dest = 'train/mix/' + name
    !mv $vocal_src $vocal_dest
    !mv $acc_src $acc_dest
    !mv $mix_src $mix_dest
    

In [ ]:
for i in validation_items:
    name = 'sample_' + str(i) + '.wav'
    vocal_src = 'vocals/' + name
    vocal_dest = 'val/vocals/' + name
    acc_src = 'acc/' + name
    acc_dest = 'val/acc/' + name
    mix_src = 'mix/' + name
    mix_dest = 'val/mix/' + name
    print(vocal_src + ' // ' + vocal_dest)
    !mv $vocal_src $vocal_dest
    !mv $acc_src $acc_dest
    !mv $mix_src $mix_dest
    

In [ ]:

random_state = 123
# create foreground folder
fg_folder = Path('~/data/audio_data/train/train').expanduser()  
fg_folder.mkdir(parents=True, exist_ok=True)                             

# create background folder - we need to provide one even if we don't use it
bg_folder = Path('~/data/audio_data/train/empty').expanduser()

labels = ['vocals', 'acc']
# 2. Create a Scaper object
sc = scaper.Scaper(
    duration=5.0,
    fg_path=str(fg_folder),
    bg_path=str(bg_folder),
    random_state=random_state
)

# 3. Set sample rate, reference dB, and channels (mono)
sc.sr = 44100
sc.ref_db = -20
sc.n_channels = 1

# 4. Define a template of probabilistic event parameters
event_parameters= {
    'label': ('const', 'vocals'),
    'source_file': ('choose', []),
    'source_time': ('uniform', 0, 1000),
    'event_time': ('const', 0),
    'event_duration': ('const', 10.0),
    'snr': ('uniform', -5, 5),
    'pitch_shift': ('uniform', -2, 2),
    'time_stretch': ('uniform', 0.8, 1.2)
}

# 5. Instatiate the template once to randomly choose a song,
#    a start time for the sources, a pitch shift and a time
#    stretch. These values must remain COHERENT across all stems
   
# Add a an event based on the probabilistic template
sc.add_event(**event_parameters)

# Instantiate the event to sample concrete values
event = sc._instantiate_event(sc.fg_spec[0])
    
# 6. Reset the Scaper object's event specficiation
sc.reset_fg_event_spec()
    
# 7. Replace the distributions for source time, pitch shift and
#    time stretch with the constant values we just sampled, to 
#    ensure our added events (stems) are coherent.    
#    NOTE: the source_file has also been sampled, and we'll keep
#    the sampled file to denote which song we'll be mixing.
event_parameters['source_time'] = ('const', event.source_time)
event_parameters['pitch_shift'] = ('const', event.pitch_shift)
event_parameters['time_stretch'] = ('const', event.time_stretch)

# 8. Iterate over the four stems (vocals, drums, bass, other) and 
#    add COHERENT events.
labels = labels

for label in labels:

    # Set the label to the stem we are adding
    event_parameters['label'] = ('const', label)

    # To ensure coherent source files (all from the same song), we leverage
    # the fact that all the stems from the same song have the same filename.
    # All we have to do is replace the stem file's parent folder name from "vocals" 
    # to the label we are adding in this iteration of the loop, which will give the 
    # correct path to the stem source file for this current label.
    coherent_source_file = event.source_file.replace('vocals', label)
    event_parameters['source_file'] = ('const', coherent_source_file)

    # Add the event using the modified, COHERENT, event parameters
    sc.add_event(**event_parameters)

In [ ]:
def generate_and_play(sc):

    mixture_audio, mixture_jam, annotation_list, stem_audio_list = sc.generate(fix_clipping=True)

    print("Mixture:")
    display(Audio(data=mixture_audio.T, rate=sc.sr))

    # extract the annotation data from the JAMS object
    ann = mixture_jam.annotations.search(namespace='scaper')[0]
    
    # iterate over the annotation and corresponding stem audio data
    for obs, stem_audio in zip(ann.data, stem_audio_list):
        print(f"Instrument: {obs.value['label']} at SNR: {obs.value['snr']:.2f}")
        display(Audio(data=stem_audio.T, rate=sc.sr))

In [ ]:
generate_and_play(sc)

In [ ]:
template_event_parameters = {
    'label': ('const', 'vocals'),
    'source_file': ('choose', []),
    'source_time': ('uniform', 0, 1000),
    'event_time': ('const', 0),
    'event_duration': ('const', 10.0),
    'snr': ('uniform', -5, 5),
    'pitch_shift': ('uniform', -2, 2),
    'time_stretch': ('uniform', 0.8, 1.2)
}

def incoherent(fg_folder, bg_folder, event_template, seed):
    """
    This function takes the paths to the MUSDB18 source materials, an event template, 
    and a random seed, and returns an INCOHERENT mixture (audio + annotations). 
    
    Stems in INCOHERENT mixtures may come from different songs and are not temporally
    aligned.
    
    Parameters
    ----------
    fg_folder : str
        Path to the foreground source material for MUSDB18
    bg_folder : str
        Path to the background material for MUSDB18 (empty folder)
    event_template: dict
        Dictionary containing a template of probabilistic event parameters
    seed : int or np.random.RandomState()
        Seed for setting the Scaper object's random state. Different seeds will 
        generate different mixtures for the same source material and event template.
        
    Returns
    -------
    mixture_audio : np.ndarray
        Audio signal for the mixture
    mixture_jams : np.ndarray
        JAMS annotation for the mixture
    annotation_list : list
        Simple annotation in list format
    stem_audio_list : list
        List containing the audio signals of the stems that comprise the mixture
    """
    
    # Create scaper object and seed random state
    sc = scaper.Scaper(
        duration=10.0,
        fg_path=str(fg_folder),
        bg_path=str(bg_folder),
        random_state=seed
    )
    
    # Set sample rate, reference dB, and channels (mono)
    sc.sr = 44100
    sc.ref_db = -20
    sc.n_channels = 1
    
    # Copy the template so we can change it
    event_parameters = event_template.copy()
    
    # Iterate over stem types and add INCOHERENT events
    labels = ['vocals', 'acc']
    for label in labels:
        event_parameters['label'] = ('const', label)
        sc.add_event(**event_parameters)
    
    # Return the generated mixture audio + annotations 
    # while ensuring we prevent audio clipping
    return sc.generate(fix_clipping=False)


def coherent(fg_folder, bg_folder, event_template, seed):
    """
    This function takes the paths to the MUSDB18 source materials and a random seed,
    and returns an COHERENT mixture (audio + annotations).
    
    Stems in COHERENT mixtures come from the same song and are temporally aligned.
    
    Parameters
    ----------
    fg_folder : str
        Path to the foreground source material for MUSDB18
    bg_folder : str
        Path to the background material for MUSDB18 (empty folder)
    event_template: dict
        Dictionary containing a template of probabilistic event parameters
    seed : int or np.random.RandomState()
        Seed for setting the Scaper object's random state. Different seeds will 
        generate different mixtures for the same source material and event template.
        
    Returns
    -------
    mixture_audio : np.ndarray
        Audio signal for the mixture
    mixture_jams : np.ndarray
        JAMS annotation for the mixture
    annotation_list : list
        Simple annotation in list format
    stem_audio_list : list
        List containing the audio signals of the stems that comprise the mixture
    """
        
    # Create scaper object and seed random state
    sc = scaper.Scaper(
        duration=10.0,
        fg_path=str(fg_folder),
        bg_path=str(bg_folder),
        random_state=seed
    )
    
    # Set sample rate, reference dB, and channels (mono)
    sc.sr = 44100
    sc.ref_db = -20
    sc.n_channels = 1
    
    # Copy the template so we can change it
    event_parameters = event_template.copy()    
    
    # Instatiate the template once to randomly choose a song,   
    # a start time for the sources, a pitch shift and a time    
    # stretch. These values must remain COHERENT across all stems
    sc.add_event(**event_parameters)
    event = sc._instantiate_event(sc.fg_spec[0])
    
    # Reset the Scaper object's the event specification
    sc.reset_fg_event_spec()
    
    # Replace the distributions for source time, pitch shift and 
    # time stretch with the constant values we just sampled, to  
    # ensure our added events (stems) are coherent.              
    event_parameters['source_time'] = ('const', event.source_time)
    event_parameters['pitch_shift'] = ('const', event.pitch_shift)
    event_parameters['time_stretch'] = ('const', event.time_stretch)

    # Iterate over the four stems (vocals, drums, bass, ot) and 
    # add COHERENT events.                                         
    labels = ['vocals', 'acc']
    for label in labels:
        
        # Set the label to the stem we are adding
        event_parameters['label'] = ('const', label)
        
        # To ensure coherent source files (all from the same song), we leverage
        # the fact that all the stems from the same song have the same filename.
        # All we have to do is replace the stem file's parent folder name from "vocals" 
        # to the label we are adding in this iteration of the loop, which will give the 
        # correct path to the stem source file for this current label.
        coherent_source_file = event.source_file.replace('vocals', label)
        event_parameters['source_file'] = ('const', coherent_source_file)
        # Add the event using the modified, COHERENT, event parameters
        sc.add_event(**event_parameters)
    
    # Generate and return the mixture audio, stem audio, and annotations
    return sc.generate(fix_clipping=False)

In [ ]:
#fg_folder = Path('~/data/audio_data/train/val').expanduser() 
for seed in [np.random.randint(0,100000)]:
    
    mixture_audio, mixture_jam, annotation_list, stem_audio_list = coherent(
        fg_folder, 
        bg_folder, 
        template_event_parameters, 
        seed)
    ann = mixture_jam.annotations.search(namespace='scaper')[0]

    for obs, stem_audio in zip(ann.data, stem_audio_list):
        print(f"Instrument: {obs.value['label']} at SNR: {obs.value['snr']:.2f}")
        if obs.value['label'] == 'vocals':
            vocals = stem_audio
        if obs.value['label'] == 'acc':
            acc = stem_audio
vocals.shape

In [ ]:
import librosa

In [ ]:
from tqdm import tqdm
bg_folder = Path('~/data/audio_data/train/empty').expanduser()
augmented_path = '/nfs/home/pedro.lopes/data/audio_data/train/augmented/'
SAMPLE_RATE=44100
start = 0
N_total = 300000
validation_size = 0.1
N_val = int(validation_size*(N_total-start))
N_train = N_total - (N_val + start)
coherent_p = 0.8
training_range = range(start,start + N_train)
validation_range = range(start+N_train,N_total,1)

fg_folder = Path('~/data/audio_data/train/train/clean').expanduser()  
fg_folder.mkdir(parents=True, exist_ok=True)   
# for i in tqdm(training_range):
#     seed = np.random.randint(0,1000000)
#     if np.random.uniform()<=coherent_p:
#         mixture_audio, mixture_jam, annotation_list, stem_audio_list = coherent(
#             fg_folder, 
#             bg_folder, 
#             template_event_parameters, 
#             seed)
#     else:
#         mixture_audio, mixture_jam, annotation_list, stem_audio_list = incoherent(
#             fg_folder, 
#             bg_folder, 
#             template_event_parameters, 
#             seed)
#     ann = mixture_jam.annotations.search(namespace='scaper')[0]
#     for obs, stem_audio in zip(ann.data, stem_audio_list):
#         if obs.value['label'] == 'vocals':
#             vocals = stem_audio
            
#     vocal_wav_encode = tf.audio.encode_wav(vocals,SAMPLE_RATE)
#     tf.io.write_file(
#         augmented_path + 'train/sample_' + str(i) + '_vocals.wav', vocal_wav_encode, name=None
#     )
#     mix_wav_encode = tf.audio.encode_wav(mixture_audio,SAMPLE_RATE)
#     tf.io.write_file(
#         augmented_path + 'train/sample_' + str(i) + '_mix.wav', mix_wav_encode, name=None
#     )
#     #librosa.output.write_wav(augmented_path + 'train/sample_' + str(i) + '_mix.wav',y=mixture_audio,sr=SAMPLE_RATE)
#     #librosa.output.write_wav(augmented_path + 'train/sample_' + str(i) + '_vocals.wav',y=vocals.T,sr=SAMPLE_RATE)
    
fg_folder = Path('~/data/audio_data/train/val/clean').expanduser()  
fg_folder.mkdir(parents=True, exist_ok=True)                             
for i in tqdm(validation_range):
    seed = np.random.randint(0,1000000)
    if np.random.uniform()<=coherent_p:
        mixture_audio, mixture_jam, annotation_list, stem_audio_list = coherent(
            fg_folder, 
            bg_folder, 
            template_event_parameters, 
            seed)
    else:
        mixture_audio, mixture_jam, annotation_list, stem_audio_list = incoherent(
            fg_folder, 
            bg_folder, 
            template_event_parameters, 
            seed)
    ann = mixture_jam.annotations.search(namespace='scaper')[0]
    for obs, stem_audio in zip(ann.data, stem_audio_list):
        if obs.value['label'] == 'vocals':
            vocals = stem_audio
    vocal_wav_encode = tf.audio.encode_wav(vocals,SAMPLE_RATE)
    tf.io.write_file(
        augmented_path + 'val/sample_' + str(i) + '_vocals.wav', vocal_wav_encode, name=None
    )
    mix_wav_encode = tf.audio.encode_wav(mixture_audio,SAMPLE_RATE)
    tf.io.write_file(
        augmented_path + 'val/sample_' + str(i) + '_mix.wav', mix_wav_encode, name=None
    )
#     librosa.output.write_wav(augmented_path + 'val/sample_' + str(i) + '_mix.wav',y=mixture_audio,sr=SAMPLE_RATE)
#     librosa.output.write_wav(augmented_path + 'val/sample_' + str(i) + '_vocals.wav',y=vocals.T,sr=SAMPLE_RATE)

In [ ]:
import matplotlib.pyplot as plt
i=3002
y,sr = librosa.core.load(augmented_path + 'train/sample_' + str(i) + '_vocals.wav',sr=None)
Audio(y,rate=sr)

In [ ]:
from utils import coherent
mixture_audio, mixture_jam, annotation_list, stem_audio_list = coherent(
            fg_folder, 
            bg_folder, 
            template_event_parameters, 
            seed)

In [ ]:
from tqdm import tqdm


In [ ]:
N_total = 50000
num_in_dir = 100
validation_size = 0.1
N_val = int(validation_size*N_total)
N_train = N_total - N_val
coherent_p = 0.8
training_range = range(0,N_train)
validation_range = range(N_train,N_total,1)
%cd /nfs/home/pedro.lopes/data/audio_data/train/augmented/train
curr_dir = 0
sample_num = 0
num_dir = int(N_train/num_in_dir)
for curr_dir in tqdm(range(1,num_dir+1)):
    if not(path.isdir(str(curr_dir))):
        dir_name = str(curr_dir) + '/'
        !mkdir $dir_name
    for i in range(num_in_dir):
        sample_num+=1
        orig_path_mix = 'sample_' + str(sample_num) + '_mix.wav'
        orig_path_vocals = 'sample_' + str(sample_num) + '_vocals.wav'
        dest_path_mix = str(curr_dir) + '/sample_' + str(sample_num) + '_mix.wav'
        dest_path_vocals = str(curr_dir) + '/sample_' + str(sample_num) + '_vocals.wav'
        !mv $orig_path_mix $dest_path_mix
        !mv $orig_path_vocals $dest_path_vocals

In [ ]:
N_total = 50000
num_in_dir = 100
validation_size = 0.1
N_val = int(validation_size*N_total)
N_train = N_total - N_val
coherent_p = 0.8
training_range = range(0,N_train)
validation_range = range(N_train,N_total,1)
%cd /nfs/home/pedro.lopes/data/audio_data/train/augmented/val
curr_dir = 0
sample_num = 45000
num_dir = int(N_val/num_in_dir)
while tqdm(curr_dir < num_dir):
    curr_dir+=1
    if not(path.isdir(curr_dir)):
        dir_name = str(curr_dir) + '/'
        !mkdir $dir_name
    for i in range(num_in_dir):
        orig_path_mix = 'sample_' + str(sample_num) + '_mix.wav'
        orig_path_vocals = 'sample_' + str(sample_num) + '_vocals.wav'
        dest_path_mix = str(curr_dir) + '/sample_' + str(sample_num) + '_mix.wav'
        dest_path_vocals = str(curr_dir) + '/sample_' + str(sample_num) + '_vocals.wav'
        !mv $orig_path_mix $dest_path_mix
        !mv $orig_path_vocals $dest_path_vocals
        sample_num+=1

In [ ]:
from glob import glob
pattern = "/home/pedro.lopes/data/audio_data/train/augmented/train/*/*/*"
names = glob(pattern)
names_corrected =[]
for name in names:
    splitted = name.rsplit('_',maxsplit=1)[0]
    names_corrected.append(splitted)
training_range = list(dict.fromkeys(names_corrected))

pattern = "/home/pedro.lopes/data/audio_data/train/augmented/val/*/*/*"
names = glob(pattern)
names_corrected =[]
for name in names:
    splitted = name.rsplit('_',maxsplit=1)[0]
    names_corrected.append(splitted)
validation_range = list(dict.fromkeys(names_corrected))

In [ ]:
training_range

In [ ]:
pattern = "/home/pedro.lopes/data/audio_data/train/train/vocals/*"
training_range = glob(pattern)
training_range

In [ ]:

audio = af.read(training_range[500] + '_vocals.wav')
ipd.Audio(audio[0],rate=44100)

In [ ]:

train_clean

In [ ]:
pattern = "/home/pedro.lopes/data/audio_data/train/val/mix/*"
training_range = glob(pattern)
mir_1k_items = ['sample_' + str(x) + '.wav' for x in range(150,1150,1)]
train_clean = [x for x in training_range if x.rsplit('/')[-1] not in mir_1k_items]
for item in train_clean:
    dest = "/home/pedro.lopes/data/audio_data/train/val/clean/mix/" + item.rsplit('/')[-1]
    !mv $item $dest
    